In [1]:
include("utils.jl")

using LightGraphs
using StatsBase
using ProgressBars

In [2]:
include("execution.jl")
include("deffuant.jl")
include("bias.jl")
include("media.jl")
include("analysis.jl")

using CSV
using DataFrames
using JSON
using LightGraphs

# Graph initialization
n = 10
p = 1.0
g = erdos_renyi(n, p)

# simulation global parameters
max_t = 100
nsteady = 10

media_op = [0.05, 0.5, 0.95]
mos = join([string(el) for el in media_op], ";", ";")

"0.05;0.5;0.95"

In [3]:
res = []
# shared opinion arrays
old_opinions = Array{Float16}(undef, nv(g))
new_opinions = Array{Float16}(undef, nv(g))
for u in vertices(g)
    op = rand()
    old_opinions[u] = Float16(op)
    new_opinions[u] = Float16(op)
end

media_op = [Float16(o) for o in media_op]

st = 0
new_opinions = copy(old_opinions)

10-element Vector{Float16}:
 0.6694
 0.762
 0.817
 0.6157
 0.2378
 0.2722
 0.0387
 0.6426
 0.1984
 0.9316

In [6]:
u = 1
uₒ = Float16(old_opinions[u])

Float16(0.6694)

In [7]:
media_access = rand()

0.11053674260244173

In [8]:
o = uₒ

Float16(0.6694)

In [ ]:
#media = [0.05, 0.5, 0.95]

In [13]:
if media_access < 0.2
    # biased access to media
    media_within_bound = [abs(uₒ - x) <= 0.2 for x in media_op]
    println("media within bound: $media_within_bound")
    candidate_media = media_op[media_within_bound]
    println("candidate $candidate_media")
    # target media selection
    if size(candidate_media)[1] > 0
        mediaₚϵ = [abs(uₒ - x)^2.0 for x in candidate_media]
        mediaₚϵ /= sum(mediaₚϵ)
        println("selection probability acc to bias $mediaₚϵ")
        selected = sample(candidate_media, Weights(mediaₚϵ))
        println("selected media $selected")
        o = (selected + uₒ) / 2
        println("new opinion $o")
    end
end

media within bound: Bool[0, 1, 0]
candidate Float16[0.5]
selection probability acc to bias [1.0]
selected media 0.5
new opinion 0.585


In [14]:
# computing neighbors
Γ = neighbors(g, u)
# obtaining neighbors' opinions
Γₚ = [old_opinions[x] for x in Γ]
println("neighbors opinions $Γₚ")


neighbors opinions Float16[0.762, 0.817, 0.6157, 0.2378, 0.2722, 0.0387, 0.6426, 0.1984, 0.9316]


In [16]:
# confidence bound filtering - by ϵ
# within_bound = [abs(uₒ - x) <= ϵ for x in Γₚ] #però se io mi sono già spostato x interazione con media non va bene devo già considerare o nel calcolo dei neighbors within_bound
within_bound = [abs(o - x)<= 0.2 for x in Γₚ]
println("neighbors within bound $within_bound")

neighbors within bound Bool[1, 0, 1, 0, 0, 0, 1, 0, 0]


In [17]:
likeminded = Γ[within_bound]

3-element Vector{Int64}:
 2
 4
 8

In [25]:
# biased sample by γ
# Γₚϵ = [abs(uₒ - x)^γ for x in old_opinions[likeminded]] #anche qua devo già considerare o dopo interazione con media
Γₚϵ = [abs(o - x)^(-2.0) for x in old_opinions[likeminded]]

3-element Vector{Float64}:
   31.830734087683748
 1056.7659360040313
  301.2283826486642

In [26]:
Γₚϵ /= sum(Γₚϵ)

3-element Vector{Float64}:
 0.0229026912595378
 0.7603589631086008
 0.2167383456318613

In [22]:
selected = sample(likeminded, Weights(Γₚϵ))

2

In [23]:
nn = Float16((old_opinions[selected] + o) / 2)

Float16(0.674)

In [24]:
new_opinions[u] = nn

Float16(0.674)